In [17]:
from math import comb,log
from mpmath import mp
import plotly.graph_objects as go
import plotly.express as px

$$ Pr(H_\text{Fair} | E) = \frac{Pr(E | H_\text{Fair}) Pr(H_\text{Fair})}{Pr(E)} $$

$$ Pr(H_\text{Fair} | E_1, E_2, \dots, E_c) = \frac{Pr(E_1 | H_\text{Fair}) Pr(E_2| H_\text{Fair}) \dots Pr(E_c | H_\text{Fair}) Pr(H_\text{Fair})}{Pr(E_1, E_2, \dots, E_c)} $$

$$ Pr(H_\text{Fair} | E_1, E_2, \dots, E_c) = \frac{\int_{p=0}^{1} Pr(E_1 | p) Pr(E_2 | p) \dots Pr(E_c | p) Pr(p | H_\text{Fair}) Pr(H_\text{Fair}) }{\int_{p=0}^{1} Pr(E_1, E_2, \dots, E_c | p)} $$

$$ \frac{Pr(H_\text{Cheating} | E_1, E_2, \dots, E_c)}{Pr(H_\text{Fair} | E_1, E_2, \dots, E_c)} = \frac{\int_{p=0}^{1} Pr(E_1 | p) Pr(E_2 | p) \dots Pr(E_c | p) Pr(p | H_\text{Cheating}) Pr(H_\text{Cheating}) }{\int_{p=0}^{1} Pr(E_1 | p) Pr(E_2 | p) \dots Pr(E_c | p) Pr(p | H_\text{Fair}) Pr(H_\text{Fair}) } $$

$$ \text{Binom}(n,k,p) = {n \choose k} p^k (1-p)^{n-k} = \frac{n!}{k!(n-k)!} p^k (1-p)^{n-k} $$

$$ \text{NegBinom}(n,k,p) = {n-1 \choose k-1} p^k (1-p)^{n-k} = \frac{(n-1)!}{(k-1)!(n-k)!} p^k (1-p)^{n-k} $$

In [11]:
with mp.workdps(30):
    print(mp.quad( lambda p: mp.fmul( mp.binomial(9,6), mp.fmul( mp.power(p,7), mp.power(1-p,3))), (0,1) ))
    
print(7 / (11*10))

0.0636363636363636363636363636364
0.06363636363636363


In [33]:
axis = list(range(20))

fig = go.Figure( data=[go.Surface(x=axis, y=axis[1:], z=[[log(comb(a+b,a)) for a in axis[1:]] for b in axis], \
                                  name='Binomial', colorscale='Reds'),
                go.Surface(x=axis, y=axis[1:], z=[[log(comb(a+b-1,a-1)) for a in axis[1:]] for b in axis], \
                                   name='Negative Binomial', colorscale='Blues')] )

fig.update_traces(showscale=False)
fig.update_layout(title="Binomal (red) and Negative Binomial (blue) comparison", scene = dict(
                    xaxis_title='k',
                    yaxis_title='n - k',
                    zaxis_title='log(Pr)'))
fig.show()

$$ \text{Beta}(\alpha,\beta,p) = \frac{\Gamma(\alpha + \beta)}{\Gamma(\alpha)\Gamma(\beta)} p^{\alpha-1} (1-p)^{\beta-1} $$

$$ \text{Beta}(n,k,p) = \frac{\Gamma(n - k + 1 + k + 1)}{\Gamma(k + 1)\Gamma(n - k + 1)} p^{k} (1-p)^{n-k} = \frac{\Gamma(n + 2)}{\Gamma(k + 1)\Gamma(n - k + 1)} p^{k} (1-p)^{n-k} $$

$$ \text{NegBeta}(n,k,p) = \frac{\Gamma(n + 1)}{\Gamma(k)\Gamma(n - k + 1)} p^{k} (1-p)^{n-k} $$

$$ \int_{p=0}^1 \text{Beta}(\alpha,\beta,p) = \int_{p=0}^1 \frac{\Gamma(\alpha + \beta)}{\Gamma(\alpha)\Gamma(\beta)} p^{\alpha-1} (1-p)^{\beta-1} = 1 $$

$$ \int_{p=0}^1 p^{\alpha-1} (1-p)^{\beta-1} = \frac{\Gamma(\alpha)\Gamma(\beta)}{\Gamma(\alpha + \beta)} $$

$$ \text{NegBeta}(n,k,p) = \frac{\frac{\Gamma(n + 1)}{\Gamma(k)\Gamma(n - k + 1)} p^{k} (1-p)^{n-k}}{\int_{p=0}^1 \frac{\Gamma(n + 1)}{\Gamma(k)\Gamma(n - k + 1)} p^{k} (1-p)^{n-k}} = \frac{\frac{\Gamma(n + 1)}{\Gamma(k)\Gamma(n - k + 1)} p^{k} (1-p)^{n-k}}{ \frac{\Gamma(n + 1)}{\Gamma(k)\Gamma(n - k + 1)} \frac{\Gamma(k + 1)\Gamma(n - k + 1)}{\Gamma(n + 2)} } = \frac{\Gamma(n + 2)}{\Gamma(k + 1)\Gamma(n - k + 1)} p^{k} (1-p)^{n-k} $$

$$ Pr(H_\text{r} | E) \propto \int_{p=0}^1 \text{NegBinom}(n,k,p) \cdot \text{NegBeta}(n,nr,p) \cdot Pr( H_\text{r} ) $$

$$ Pr(H_\text{r} | E) \propto Pr( H_\text{r} ) \int_{p=0}^1 \frac{(n-1)!}{(k-1)!(n-k)!} p^k (1-p)^{n-k} \frac{\Gamma(n + 2)}{\Gamma(nr +1)\Gamma(n - nr + 1)} p^{nr} (1-p)^{n-nr} $$

$$ \text{NegBeta}( \vec n, r, p ) = \frac{\prod_{j=1}^{c} \frac{\Gamma(n_j + 2)}{\Gamma(n_jr + 1)\Gamma(n_j - n_jr + 1)} p^{n_jr} (1-p)^{n_j(1-r)}}{\int_{p=0}^{1} \prod_{j=1}^{c} \frac{\Gamma(n_j + 2)}{\Gamma(n_jr + 1)\Gamma(n_j - n_jr + 1)} p^{n_jr} (1-p)^{n_j(1-r)}} $$ 

$$ \text{NegBeta}( \vec n, r, p ) = \frac{\prod_{j=1}^{c} p^{n_jr} (1-p)^{n_j(1-r)} }{\int_{p=0}^{1} \prod_{j=1}^{c}  p^{n_jr} (1-p)^{n_j(1-r)}} $$ 

$$ \prod_{j=1}^{c} p^{n_jr} (1-p)^{n_j(1-r)} = p^{r\sum_{j=1}^{c} n_j} (1-p)^{(1-r)\sum_{j=1}^{c} n_j} $$

$$ \alpha = 1 + \sum_{j = 1}^{c} n_jr = 1 + r\sum_{j = 1}^{c} n_j $$ 

$$ \beta = 1 + \sum_{j = 1}^{c}n_j(1 - r) = 1 + (1-r)\sum_{j = 1}^{c}n_j $$

$$ \|\vec n\|_1 = \sqrt[1]{n_1^1 + n_2^1 + n_3^1 + \dots + n_c^1} = \sum_{j=1}^{c} n_j $$

$$ \text{NegBeta}( \vec n, r, p ) = \frac{\Gamma(1 + r\|\vec n\|_1)\Gamma(1 + (1-r)\|\vec n\|_1) p^{r\|\vec n\|_1} (1-p)^{(1-r)\|\vec n\|_1}}{\Gamma(1 + r\|\vec n\|_1 + 1 + (1-r)\|\vec n\|_1) } = \frac{\Gamma(1 + r\|\vec n\|_1)\Gamma(1 + (1-r)\|\vec n\|_1) }{\Gamma(2 + \|\vec n\|_1) } p^{r\|\vec n\|_1} (1-p)^{(1-r)\|\vec n\|_1} $$ 

$$ \text{NegBinom}(\vec n, \vec k, p) = \prod_{j=1}^{c} \frac{(n_j-1)!}{(k_j-1)!(n_j-k_j)!} p^{k_j} (1-p)^{n_j-k_j} = \prod_{j=1}^{c} \frac{\Gamma(n_j)}{\Gamma(k_j)\Gamma(n_j-k_j+1)} p^{k_j} (1-p)^{n_j-k_j} $$

$$ Pr(H_\text{r} | E_1, E_2, \dots E_c) \propto Pr( H_\text{r} ) \int_{p=0}^1\frac{\Gamma(1 + r\|\vec n\|_1)\Gamma(1 + (1-r)\|\vec n\|_1) }{\Gamma(2 + \|\vec n\|_1) } p^{r\|\vec n\|_1} (1-p)^{(1-r)\|\vec n\|_1} \prod_{j = 1}^{c} \frac{\Gamma(n_j)}{\Gamma(k_j)\Gamma(n_j-k_j+1)} p^{k_j} (1-p)^{n_j-k_j} $$

$$ Pr(H_\text{r} | E_1, E_2, \dots E_c) \propto Pr( H_\text{r} ) \frac{\Gamma(1 + r\|\vec n\|_1)\Gamma(1 + (1-r)\|\vec n\|_1) \prod_{j = 1}^{c} \Gamma(n_j) }{\Gamma(2 + \|\vec n\|_1) \prod_{j = 1}^{c} \Gamma(k_j)\Gamma(n_j-k_j+1) } \int_{p=0}^1 p^{r\|\vec n\|_1} (1-p)^{(1-r)\|\vec n\|_1} p^{\|\vec k\|_1} (1-p)^{\|\vec n\|_1-\|\vec k\|_1} $$

$$ \alpha = 1 + r\|\vec n\|_1 + \|\vec k\|_1 $$ 

$$ \beta = 1 + (1-r)\|\vec n\|_1 + \|\vec n\|_1 - \|\vec k\|_1 = 1 + (2-r)\|\vec n\|_1 - \|\vec k\|_1 $$

$$ Pr(H_\text{r} | E_1, E_2, \dots E_c) \propto Pr( H_\text{r} ) \frac{\Gamma(1 + r\|\vec n\|_1)\Gamma(1 + (1-r)\|\vec n\|_1) \prod_{j = 1}^{c} \Gamma(n_j) }{\Gamma(2 + \|\vec n\|_1) \prod_{j = 1}^{c} \Gamma(k_j)\Gamma(n_j-k_j+1) } \frac{\Gamma(1 + r\|\vec n\|_1 + \|\vec k\|_1)\Gamma(1 + (2-r)\|\vec n\|_1 - \|\vec k\|_1) }{\Gamma(1 + r\|\vec n\|_1 + \|\vec k\|_1 + 1 + (2-r)\|\vec n\|_1 - \|\vec k\|_1)} $$

$$ Pr(H_\text{r} | E_1, E_2, \dots E_c) \propto Pr( H_\text{r} ) \frac{\Gamma(1 + r\|\vec n\|_1)\Gamma(1 + (1-r)\|\vec n\|_1) \Gamma(1 + r\|\vec n\|_1 + \|\vec k\|_1)\Gamma(1 + (2-r)\|\vec n\|_1 - \|\vec k\|_1) \prod_{j = 1}^{c} \Gamma(n_j) }{\Gamma(2 + \|\vec n\|_1) \Gamma(2 + 2\|\vec n\|_1) \prod_{j = 1}^{c} \Gamma(k_j)\Gamma(n_j-k_j+1) } $$

$$ Pr( H_\text{Fair} | E_1, E_2, \dots, E_c ) = Pr( H_{r = 0.5} | E_1, E_2, \dots, E_c ) $$

$$ \frac{Pr(H_\text{Cheating} | E_1, E_2, \dots, E_c)}{Pr(H_\text{Fair} | E_1, E_2, \dots, E_c)} = \frac{\Gamma(1 + r_\text{Cheating}\|\vec n\|_1)\Gamma(1 + (1-r_\text{Cheating})\|\vec n\|_1) \Gamma(1 + r_\text{Cheating}\|\vec n\|_1 + \|\vec k\|_1)\Gamma(1 + (2-r_\text{Cheating})\|\vec n\|_1 - \|\vec k\|_1)}{\Gamma(1 + r_\text{Fair}\|\vec n\|_1)\Gamma(1 + (1-r_\text{Fair})\|\vec n\|_1) \Gamma(1 + r_\text{Fair}\|\vec n\|_1 + \|\vec k\|_1)\Gamma(1 + (2-r_\text{Fair})\|\vec n\|_1 - \|\vec k\|_1)} \frac{H_\text{Cheating}}{H_\text{Fair}} $$

In [22]:
def bayes_factor( vec_n, vec_k, r_cheating, r_fair, H_cheating=1, H_fair=1 ):
    """Calculate the Bayes Factor for a given task, specifically H_cheating / H_fair.
        Relies on mpmath to perform the calculations, which also means you can adjust the precision.
    
    Parameters
    ----------
    vec_n: A list containing the total attempts at this task until it was completed.
    vec_k: A list containing the successful attempts at this task until it was completed.
    r_cheating: A float representing the probability of success predicted by H_cheating.
    r_fair: A float representing the probability of success predicted by H_fair.
    H_cheating: A float representing the prior probability of H_cheating being true. Optional, defaults to 1.
    H_fair: A float representing the prior probability of H_fair being true. Optional, defaults to 1.
    
    Returns
    -------
    The Bayes Factor, as an mpmath float."""
    
    from mpmath import gammaprod, fmul, fdiv
    from numpy import sum        # vectorized
    
    # sanity check some incoming values
    assert (r_cheating > 0) and (r_cheating < 1)
    assert (r_fair > 0) and (r_fair < 1)
    assert len(vec_n) == len(vec_k)
    for i,n in enumerate(vec_n):
        assert n > 0
        assert vec_k[i] >= 0
        assert vec_k[i] <= n
        
    # calculate the sums first
    sum_n = sum(vec_n)
    sum_k = sum(vec_k)
    
    return fmul( gammaprod( [1 + r_cheating*sum_n, 1 + (1-r_cheating)*sum_n, 
                       1 + r_cheating*sum_n + sum_k, 1 + (2-r_cheating)*sum_n - sum_k], \
                      [1 + r_fair*sum_n, 1 + (1-r_fair)*sum_n, 
                       1 + r_fair*sum_n + sum_k, 1 + (2-r_fair)*sum_n - sum_k] ), \
                fdiv( H_cheating, H_fair ) )

$$ \text{NegBinom}(\vec n, \vec k, p) = \prod_{j=1}^{c} \frac{(n_j-1)!}{(k_j-1)!(n_j-k_j)!} p^{k_j} (1-p)^{n_j-k_j} \ $$

$$ p^{\|k\|_1} (1-p)^{\|n\|_1-\|k\|_1} $$

$$ \alpha = 1 + \|\vec k\|_1 $$ 

$$ \beta = 1 + \|\vec n\|_1 - \|\vec k\|_1 $$

$$ \frac{\alpha - 1}{\alpha + \beta - 2} = \frac{\|\vec k\|_1}{\|\vec k\|_1 + \|\vec n\|_1 - \|\vec k\|_1} = \frac{\|\vec k\|_1}{\|\vec n\|_1} $$

In [19]:
def r_maximal_likelihood( vec_n, vec_k ):
    """Calculate the maximal likelihood for a given task, which is typically used for r_cheating.
        Relies on mpmath to perform the calculations, which also means you can adjust the precision.
    
    Parameters
    ----------
    vec_n: A list containing the total attempts at this task until it was completed.
    vec_k: A list containing the successful attempts at this task until it was completed.
    
    Returns
    -------
    The Bayes Factor, as an mpmath float."""
    
    from mpmath import fdiv
    from numpy import sum        # vectorized
    
    # sanity check some incoming values
    assert len(vec_n) == len(vec_k)
    for i,n in enumerate(vec_n):
        assert n > 0
        assert vec_k[i] >= 0
        assert vec_k[i] <= n
        
    # calculate the sums
    sum_n = sum(vec_n)
    sum_k = sum(vec_k)
    
    # just do the division
    return fdiv( sum_k, sum_n )

In [36]:
from numpy.random import default_rng

random  = default_rng(42)
mean    = 8.2
stdev   = 3.6
r_cheat = 0.55
r_fair  = 0.5
count   = 100

vec_n     = list()
cheat_k = list()
cheat_y = list()
fair_k    = list()
fair_y    = list()

for _ in range(count):
    
    n = int(random.standard_normal()*stdev + mean + .5) + 1
    vec_n.append( n )
    cheat_k.append( int(r_cheat*n + .5) )
    fair_k.append( int(r_fair*n + .5) )
    
    cheat_y.append( float(bayes_factor(vec_n, cheat_k, r_maximal_likelihood(vec_n, cheat_k), r_fair)) ) 
    fair_y.append( float(bayes_factor(vec_n, fair_k, r_maximal_likelihood(vec_n, fair_k), r_fair)) )
    
x = list(range(1,count+1))
fig = go.Figure()
fig.add_trace( go.Scatter(x=x, y=cheat_y, name='cheater', mode='lines') )
fig.add_trace( go.Scatter(x=x, y=fair_y, name='fair', mode='lines') )

fig.update_layout(title=f"Bayes Factor behavior for a cheater (r={r_cheat}) vs. someone playing fair (r={r_fair}).")
fig.update_yaxes(type="log")

fig.show()

In [21]:
with mp.workdps(30):
    print(mp.gammaprod( [10+1,7+1,10-7+1], [7,10-7+1,10+2] ))

0.636363636363636363636363636364
